In [1]:
import pandas as pd
from collections import defaultdict

df = pd.read_csv(
    "desensitized_data/Desensitized_data_from_597_participants.csv",
    dtype=str  # Set all columns to string type when reading the CSV
)

# Create a dictionary to store columns with the same base name
column_groups = defaultdict(list)

for col in df.columns:
    base_name = col.split(".")[0]  # Remove any duplicated suffix (e.g., ".1", ".2")
    column_groups[base_name].append(col)

# Create a new DataFrame with merged data
df_merged = pd.DataFrame()

for base_name, col_variants in column_groups.items():
    # If there's only one column, just copy it
    if len(col_variants) == 1:
        df_merged[base_name] = df[col_variants[0]]
    else:
        # Merge multiple columns by taking the first non-null value
        df_merged[base_name] = df[col_variants].bfill(axis=1).iloc[:, 0]

df = df_merged.copy()
df_merged

,Start Date,End Date,Response Type,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,User Language,Q_RecaptchaScore,...,"Ability to respond quickly to control signals, and to apply great force smoothly and precisely","Ability to perform repetitive, routine tasks",Ability to reason inductively,Ability to detect small amounts of visual or acoustic energy,Ability to perceive patterns of light or sound,Ability to store very large amounts of information for long periods and to recall relevant facts at the appropriate time,Ability to store information briefly and then to erase it completely,"Ability to reason deductively, including computational ability",Please provide any suggestions or comments on the survey,Group
0,2/12/2025 22:24,2/12/2025 22:32,IP Address,100,510,True,2/12/2025 22:32,R_3eqq7Gofo6sZ3Tk,EN,1,...,Machines surpass humans,Machines surpass humans,Machines surpass humans,Humans surpass machines,Machines surpass humans,Machines surpass humans,Machines surpass humans,Machines surpass humans,But it all depends on human programming.,3
1,2/12/2025 22:31,2/12/2025 22:35,IP Address,100,228,True,2/12/2025 22:35,R_5J8uHvrhM4sRIjL,EN,1,...,Machines surpass humans,Machines surpass humans,Humans surpass machines,Humans surpass machines,Humans surpass machines,Machines surpass humans,Machines surpass humans,Machines surpass humans,NaN,2
2,2/12/2025 22:29,2/12/2025 22:35,IP Address,100,363,True,2/12/2025 22:35,R_6OGEs9Jk37UWjM2,EN,1,...,Machines surpass humans,Machines surpass humans,Humans surpass machines,Machines surpass humans,Machines surpass humans,Machines surpass humans,Machines surpass humans,Humans surpass machines,NaN,1
3,2/12/2025 22:27,2/12/2025 22:36,IP Address,100,485,True,2/12/2025 22:36,R_7EbCXnJ4ZDFVQc9,EN,1,...,Machines surpass humans,Machines surpass humans,Humans surpass machines,Machines surpass humans,Machines surpass humans,Machines surpass humans,Machines surpass humans,Machines surpass humans,No issues with this survey! Thank you,2
4,2/12/2025 22:27,2/12/2025 22:36,IP Address,100,516,True,2/12/2025 22:36,R_8FWlrwL9VWbkqlK,EN,1,...,Machines surpass humans,Machines surpass humans,Humans surpass machines,Machines surpass humans,Machines surpass humans,Machines surpass humans,Machines surpass humans,Machines surpass humans,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,2/13/2025 0:00,2/13/2025 0:06,IP Address,100,367,True,2/13/2025 0:06,R_8cUBTXrjeEqbwfD,EN,1,...,Machines surpass humans,Humans surpass machines,Humans surpass machines,Machines surpass humans,Machines surpass humans,Machines surpass humans,Humans surpass machines,Machines surpass humans,Everything worked well and the instructions we...,2
593,2/13/2025 0:01,2/13/2025 0:09,IP Address,100,488,True,2/13/2025 0:09,R_8Hk1ZYuWBC6z3dF,EN,1,...,Humans surpass machines,Machines surpass humans,Humans surpass machines,Machines surpass humans,Machines surpass humans,Machines surpass humans,Humans surpass machines,Machines surpass humans,NaN,2
594,2/13/2025 0:07,2/13/2025 0:12,IP Address,100,314,True,2/13/2025 0:12,R_3D0NwWA1A4pE3tL,EN,1,...,Machines surpass humans,Machines surpass humans,Humans surpass machines,Humans surpass machines,Machines surpass humans,Machines surpass humans,Machines surpass humans,Humans surpass machines,NaN,1
595,2/13/2025 0:16,2/13/2025 0:31,IP Address,100,867,True,2/13/2025 0:31,R_2ozlRcIZoDwzO38,EN,1,...,I prefer not to respond,I prefer not to respond,I prefer not to respond,I prefer not to respond,I prefer not to respond,I prefer not to respond,I prefer not to respond,I prefer not to respond,No comment,2


In [2]:
import csv

def save_q7_responses(df_merged, include_pid=True):
    # Display column names to identify the correct Q7-related questions
    q7_related_columns = [
        col for col in df_merged.columns if "10 years" in col or "replacement" in col
    ]

    # Identifying Group column
    group_column = "Group"
    res_id_column = "Response ID"

    group_files = {
        "1": "desensitized_data/Scenario_A_Q7.csv",
        "2": "desensitized_data/Scenario_B_Q7.csv",
        "3": "desensitized_data/Scenario_C_Q7.csv",
    }
    # Extract Q7 responses and save to CSV based on group
    saved_files = {}

    for group_id, file_path in group_files.items():
        # Selecting the corresponding Q7 column based on group
        scenario_index = int(group_id) - 1
        q7_column = q7_related_columns[scenario_index]

        # Extracting data for the given group
        if include_pid:
            df_group = df_merged[df_merged[group_column] == group_id][
                [res_id_column, q7_column]
            ]
        else:
            df_group = df_merged[df_merged[group_column] == group_id][[q7_column]]

        # Renaming Q7 column for clarity
        df_group.rename(columns={q7_column: "Q7_Response"}, inplace=True)

        # Saving to CSV
        df_group.to_csv(file_path, index=False, quoting=csv.QUOTE_ALL)
        saved_files[group_id] = file_path
        print(len(df_group))

    return saved_files


# Example usage:
saved_files = save_q7_responses(df_merged, include_pid=True)

195
211
191


In [3]:
def save_responses(df_merged, save_all_columns=True):
    # Define the questions for Q5 and Q6
    q5 = "Comment on the technical quality of the robot as shown in the video"
    q6 = "Do you have any recommendations about how to improve the robot?"

    # Identifying Group column
    group_column = "Group"
    res_id_column = "Response ID"

    # Define file paths for Q5 and Q6 responses
    group_files_q5 = {
        "1": "desensitized_data/Scenario_A_Q5.csv",
        "2": "desensitized_data/Scenario_B_Q5.csv",
        "3": "desensitized_data/Scenario_C_Q5.csv",
    }

    group_files_q6 = {
        "1": "desensitized_data/Scenario_A_Q6.csv",
        "2": "desensitized_data/Scenario_B_Q6.csv",
        "3": "desensitized_data/Scenario_C_Q6.csv",
    }

    # Extract Q5 and Q6 responses and save to CSV based on group
    saved_files_q5 = {}
    saved_files_q6 = {}

    for group_id, file_path in group_files_q5.items():
        # Extracting data for Q5 for the given group
        if save_all_columns:
            df_group_q5 = df[df[group_column] == group_id][
                [res_id_column, q5, group_column]
            ]
        else:
            df_group_q5 = df[df[group_column] == group_id][[q5]]
        df_group_q5.rename(columns={q5: "Q5_Response"}, inplace=True)
        df_group_q5.to_csv(file_path, index=False, quoting=csv.QUOTE_ALL)
        saved_files_q5[group_id] = file_path
        print(
            f"Saved {len(df_group_q5)} Q5 responses for group {group_id} to {file_path}"
        )

    for group_id, file_path in group_files_q6.items():
        # Extracting data for Q6 for the given group
        if save_all_columns:
            df_group_q6 = df[df[group_column] == group_id][
                [res_id_column, q6, group_column]
            ]
        else:
            df_group_q6 = df[df[group_column] == group_id][[q6]]
        df_group_q6.rename(columns={q6: "Q6_Response"}, inplace=True)
        df_group_q6.to_csv(file_path, index=False, quoting=csv.QUOTE_ALL)
        saved_files_q6[group_id] = file_path
        print(
            f"Saved {len(df_group_q6)} Q6 responses for group {group_id} to {file_path}"
        )

    return saved_files_q5, saved_files_q6


# Example usage:
saved_files_q5, saved_files_q6 = save_responses(df_merged, save_all_columns=True)

Saved 195 Q5 responses for group 1 to desensitized_data/Scenario_A_Q5.csv
Saved 211 Q5 responses for group 2 to desensitized_data/Scenario_B_Q5.csv
Saved 191 Q5 responses for group 3 to desensitized_data/Scenario_C_Q5.csv
Saved 195 Q6 responses for group 1 to desensitized_data/Scenario_A_Q6.csv
Saved 211 Q6 responses for group 2 to desensitized_data/Scenario_B_Q6.csv
Saved 191 Q6 responses for group 3 to desensitized_data/Scenario_C_Q6.csv


In [5]:
import pandas as pd

# Define file paths for the three CSV files
file_paths = {
    "Robotics Education": ["desensitized_data/Scenario_A_Q7_o3.csv", "desensitized_data/Scenario_A_Q7_Gemini.csv"],
    "Chess Coach": ["desensitized_data/Scenario_B_Q7_o3.csv", "desensitized_data/Scenario_B_Q7_Gemini.csv"],
    "Home Entertainment": ["desensitized_data/Scenario_C_Q7_o3.csv", "desensitized_data/Scenario_C_Q7_Gemini.csv"],
}

# Create an empty list to store merged DataFrames
merged_dfs = []

# Process each scenario
for scenario_name, paths in file_paths.items():
    # Read the two CSV files for the scenario
    df_o = pd.read_csv(paths[0])
    df_g = pd.read_csv(paths[1])

    # Rename the Category columns
    df_o.rename(columns={"Category": "Category_o"}, inplace=True)
    df_g.rename(columns={"Category": "Category_G"}, inplace=True)

    # Merge the DataFrames on Response ID and Q7_Response
    merged_df = pd.merge(df_o, df_g, on=["Response ID"])
    merged_df.drop("Q7_Response_y", axis=1, inplace=True)
    merged_df.rename(columns={"Q7_Response_y": "Q7_Response"}, inplace=True)

    # Add Agreed and Category_Final columns
    merged_df["Agreed"] = merged_df["Category_o"] == merged_df["Category_G"]
    merged_df["Category_Final"] = merged_df.apply(
        lambda row: row["Category_o"] if row["Agreed"] else None, axis=1
    )

    # Add the Group column based on the scenario
    group_value = "1" if "A" in scenario_name else "2" if "B" in scenario_name else "3"
    merged_df["Group"] = group_value

    # Add the merged DataFrame to the list
    merged_dfs.append(merged_df)

# Concatenate all merged DataFrames into a single DataFrame
final_df = pd.concat(merged_dfs, ignore_index=True)

import csv

final_df.to_csv("desensitized_data/Q7_annotated_three_scenarios_llms.csv", index=False, quoting=csv.QUOTE_ALL)
final_df

,Response ID,Q7_Response_x,Category_o,Category_G,Agreed,Category_Final,Group
0,R_6OGEs9Jk37UWjM2,"Yes, I think robots could be helpful and serve...",positive,positive,True,positive,3
1,R_8FWlrwL9VWbkqlK,"It's possible, but I would need to see code al...",neutral,neutral,True,neutral,3
2,R_7QzNPdA4ydCELbH,Maybe in some scenarios but not all. I think i...,neutral,neutral,True,neutral,3
3,R_5Kxqw6pc9qG2EH5,"Possibly, but I think people could benefit fro...",neutral,neutral,True,neutral,3
4,R_3LKiyRiEJRSupO1,I do. I feel that their are various situations...,positive,positive,True,positive,3
...,...,...,...,...,...,...,...
592,R_3ITm4a9ZyNUoeIh,"No, television is here to stay. The quality of...",negative,negative,True,negative,3
593,R_8zwGhfwBJ6MN3Nb,No I don't think so. Perhaps the robot could h...,negative,negative,True,negative,3
594,R_7QlrwNG9tEAo7hn,Totally not. It's just a less efficient way to...,negative,negative,True,negative,3
595,R_25A54qUELgAGZQx,"No, more human interaction is preferred.",negative,negative,True,negative,3
